In [15]:
import cv2
import numpy as np
import mediapipe as mp
import matplotlib.pyplot as plt

In [16]:
#skeleton 추출 준비
mp_hands = mp.solutions.hands


hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.3)


mp_drawing = mp.solutions.drawing_utils

In [27]:
from PIL import Image
import pandas as pd
import os
import cv2
#손동작 pp image list 생성
image_path_test='./total_pp-1/test/images/'
label_path_test='./total_pp-1/test/labels/'
image_path_train='./total_pp-1/train/images/'
label_path_train='./total_pp-1/train/labels/'
image_path_valid='./total_pp-1/valid/images/'
label_path_valid='./total_pp-1/valid/labels/'

image_path=[image_path_test,image_path_train,image_path_valid]
label_path=[label_path_test,label_path_train,label_path_valid]

image_list_test=os.listdir(image_path_test)
image_list_train=os.listdir(image_path_train)
image_list_valid=os.listdir(image_path_valid)

label_list_test=os.listdir(label_path_test)
label_list_train=os.listdir(label_path_train)
label_list_valid=os.listdir(label_path_valid)

image_path_list=[image_list_test,image_list_train,image_list_valid]
label_path_list=[label_list_test,label_list_train,label_list_valid]


In [17]:
from PIL import Image
import pandas as pd
import os
import cv2
#손동작 cnv image list 생성
image_path_test='./cnv_total-2/test/images/'
label_path_test='./cnv_total-2/test/labels/'
image_path_train='./cnv_total-2/train/images/'
label_path_train='./cnv_total-2/train/labels/'
image_path_valid='./cnv_total-2/valid/images/'
label_path_valid='./cnv_total-2/valid/labels/'

image_path=[image_path_test,image_path_train,image_path_valid]
label_path=[label_path_test,label_path_train,label_path_valid]

image_list_test=os.listdir(image_path_test)
image_list_train=os.listdir(image_path_train)
image_list_valid=os.listdir(image_path_valid)

label_list_test=os.listdir(label_path_test)
label_list_train=os.listdir(label_path_train)
label_list_valid=os.listdir(label_path_valid)

image_path_list=[image_list_test,image_list_train,image_list_valid]
label_path_list=[label_list_test,label_list_train,label_list_valid]


In [18]:
# 각 이미지 마다 object detection 결과값에 맞춰 사진을 자르고 skeleton을 추출하여 csv 파일에 저장
keypoint_data=list()
for i in range(len(image_path_list)):
    for j in range(len(image_path_list[i])):
        image=cv2.imread(image_path[i]+image_path_list[i][j])
        label=open(label_path[i]+label_path_list[i][j])

        line=label.readline().split()
        #line 마다 x.center,y.center,width,height 순으로 적혀있고 픽셀 기준으로 적혀 있기 떄문에 거기에 맞춰 계산
        x1=int((float(line[1])-float(line[3])/2)*640)
        x2=int((float(line[1])+float(line[3])/2)*640)
        y1=int((float(line[2])-float(line[4])/2)*640)
        y2=int((float(line[2])+float(line[4])/2)*640)
        #object detection에 맞춰 이미지 자르기
        croppedImage1=image[y1:y2,x1:x2]

        class1=line[0]

        line=label.readline().split()
        #line 마다 x.center,y.center,width,height 순으로 적혀있고 픽셀 기준으로 적혀 있기 떄문에 거기에 맞춰 계산
        x1=int((float(line[1])-float(line[3])/2)*640)
        x2=int((float(line[1])+float(line[3])/2)*640)
        y1=int((float(line[2])-float(line[4])/2)*640)
        y2=int((float(line[2])+float(line[4])/2)*640)
        #object detection에 맞춰 이미지 자르기
        croppedImage2=image[y1:y2,x1:x2]
        class2=line[0]

        results1 = hands.process(cv2.cvtColor(croppedImage1, cv2.COLOR_BGR2RGB))
        results2 = hands.process(cv2.cvtColor(croppedImage2, cv2.COLOR_BGR2RGB))
    
        #skeleton 추출,왼속 오른손 총 2개
        if results1.multi_hand_landmarks:

            for hand_no, hand_landmarks in enumerate(results1.multi_hand_landmarks):
                sample=list()
                sample.append(class1)
                for j in range(21):
                    sample.append(hand_landmarks.landmark[mp_hands.HandLandmark(j).value].x)
                    sample.append(hand_landmarks.landmark[mp_hands.HandLandmark(j).value].y)
                    sample.append(hand_landmarks.landmark[mp_hands.HandLandmark(j).value].z)
                # print(sample)
                keypoint_data.append(sample)
        if results2.multi_hand_landmarks:
        
            for hand_no, hand_landmarks in enumerate(results2.multi_hand_landmarks):
                sample=list()
                sample.append(class2)
                for l in range(21):
                    sample.append(hand_landmarks.landmark[mp_hands.HandLandmark(l).value].x)
                    sample.append(hand_landmarks.landmark[mp_hands.HandLandmark(l).value].y)
                    sample.append(hand_landmarks.landmark[mp_hands.HandLandmark(l).value].z)
                keypoint_data.append(sample)

In [19]:
#skeleton point마다 좌표 값을 pandas로 정리
df=pd.DataFrame(keypoint_data,columns=['label','WRIST_X','WRIST_Y','WRIST_Z',
                                       'THUMB_CMC_X','THUMB_CMC_Y','THUMB_CMC_Z',
                                       'THUMB_MCP_X','THUMB_MCP_Y','THUMB_MCP_Z',
                                       'THUMB_IP_X','THUMB_IP_Y','THUMB_IP_Z',
                                       'THUMB_TIP_X','THUMB_TIP_Y','THUMB_TIP_Z',
                                       'INDEX_FINGER_MCP_X','INDEX_FINGER_MCP_Y','INDEX_FINGER_MCP_Z',
                                       'INDEX_FIGNER_PIP_X','INDEX_FIGNER_PIP_Y','INDEX_FIGNER_PIP_Z',
                                       'INDEX_FINGER_DIP_X','INDEX_FINGER_DIP_Y','INDEX_FINGER_DIP_Z',
                           'INDEX_FINGER_TIP_X','INDEX_FINGER_TIP_Y','INDEX_FINGER_TIP_Z',
                                       'MIDDLE_FINGER_MCP_X','MIDDLE_FINGER_MCP_Y','MIDDLE_FINGER_MCP_Z',
                                       'MIDDLE_FINGER_PIP_X','MIDDLE_FINGER_PIP_Y','MIDDLE_FINGER_PIP_Z',
                                       'MIDDLE_FINGER_DIP_X','MIDDLE_FINGER_DIP_Y','MIDDLE_FINGER_DIP_Z',
                                       'MIDDLE_FINGER_TIP_X','MIDDLE_FINGER_TIP_Y','MIDDLE_FINGER_TIP_Z',
                                       'RING_FINGER_MCP_X','RING_FINGER_MCP_Y','RING_FINGER_MCP_Z',
                                    'RING_FINGER_PIP_X','RING_FINGER_PIP_Y','RING_FINGER_PIP_Z',
                                       'RING_FINGER_DIP_X','RING_FINGER_DIP_Y','RING_FINGER_DIP_Z',
                                       'RING_FIGNER_TIP_X','RING_FINGER_TIP_Y','RING_FINGER_TIP_Z',
                                       'PINKY_MCP_X','PINKY_MCP_Y','PINKY_MCP_Z',
                                       'PINKY_PIP_X','PINKY_PIP_Y','PINKY_PIP_Z',
                                       'PINKY_DIP_X','PINKY_DIP_Y','PINKY_DIP_Z',
                           'PINKY_TIP_X','PINKY_TIP_Y','PINKY_TIP_Z'])

In [20]:
#csv 파일로 저장
df.to_csv('./data/cnv_total-2_aug_keypoint.csv',index=None)